In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import numpy as np # linear algebra
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import GlobalMaxPooling1D, Conv1D, Dense, Embedding, Dropout,MaxPooling1D, LSTM
import matplotlib.pyplot as plt
import numpy as np # linear algebra
from keras.callbacks import EarlyStopping

from keras import regularizers
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report, f1_score
import os
for dirname, _, filenames in os.walk('../input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import tensorflow 
from tensorflow.python.lib.io import file_io


In [ ]:
data = pd.read_csv('../input/finaldata/YouTube_Dadvar_2014.csv')

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.ndim

In [ ]:
data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
data.shape

In [ ]:
data.dtypes

In [ ]:
data.isnull().any()

In [ ]:
data.columns

In [ ]:
data_dupli = data.drop_duplicates()
data=data_dupli
data.describe()

In [ ]:
from sklearn.preprocessing import LabelEncoder
data['Class']=data['Class'].astype(str)
le = LabelEncoder()
data['Class'] = le.fit_transform(data['Class'])

In [ ]:
data['Class'].value_counts().plot(kind='barh')

In [ ]:
import kernelprocess

df = kernelprocess.processing(data)
df.head()

In [ ]:
from sklearn.utils import shuffle
file = shuffle(df, random_state=3)

In [ ]:
Y = pd.DataFrame(file['Class'])
X = pd.DataFrame(file['Comments'])

In [ ]:
def plot_model_history(model_history):
        fig, axs = plt.subplots(1,2,figsize=(15,5))
        # summarize history for accuracy
        axs[0].plot(range(1,len(model_history.history['acc'])+1),model_history.history['acc'])
        axs[0].plot(range(1,len(model_history.history['val_acc'])+1),model_history.history['val_acc'])
        axs[0].set_title('Model Accuracy')
        axs[0].set_xticks(np.arange(1,len(model_history.history['acc'])+1),len(model_history.history['acc'])/10)
        axs[0].legend(['train', 'test'], loc='best')
        # summarize history for loss
        axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
        axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
        axs[1].set_title('Model Loss')
        axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
        axs[1].legend(['train', 'test'], loc='best')
        plt.show()

In [ ]:
tokenizer = Tokenizer(num_words=None, split=' ',lower=False)
tokenizer.fit_on_texts(X['Comments'].values)
x = tokenizer.texts_to_sequences(X['Comments'].values)
wordindex = tokenizer.word_index
vocab_size = len(wordindex) + 1
token = pad_sequences(x, padding  = 'post', maxlen = 2303) #max comment length

In [ ]:
import keras.backend as K
def mean_pred(y_true, y_pred):
    return K.mean(y_pred)


# load whole embedding into memory
print('Indexing word vectors.')
    
embeddings_index = {}
f = file_io.FileIO('../input/finaldata/glove.6B.100d.txt', mode='r')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
    
print('Found %s word vectors.' % len(embeddings_index))
#emb_dim could be a parameter
    
all_embs = np.stack(embeddings_index.values())
emb_mean = all_embs.mean() 
emb_std = all_embs.std() 
emb_mean,emb_std
emb_dim=100
embedding_matrix = np.random.normal(emb_mean, emb_std, (vocab_size, emb_dim ))
for word, i in wordindex.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
        embedding_matrix[i] = embedding_vector


In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
cv = KFold(n_splits=4, random_state=42)

In [ ]:
#def CNNmodel(x_train, x_test, y_train, y_test, vocab_size, embedding_matrix):
y=Y['Class'].values  
cvscores = []
for train, test in cv.split(token):  
    #print("TRAIN:", train, "Test:", test)
    Xtrain, Xtest = token[train], token[test]
    Ytrain, Ytest = y[train], y[test]
    #ramdomOverSample
    from imblearn.over_sampling import RandomOverSampler
    ros = RandomOverSampler()
    x_ros, y_ros = ros.fit_sample(Xtrain, Ytrain)   
    
    seed = 121
    np.random.seed(seed)
    model = Sequential()
    model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False, input_length = x_ros.shape[1]))
    model.add(LSTM(180, activation='tanh', return_sequences=True))
    #model.add(Dropout(0.4))
    model.add(LSTM(180, activation='tanh', return_sequences=True))
    model.add(Dropout(0.3))
    model.add(GlobalMaxPooling1D())
    model.add(Dropout(0.5))
    model.add(Dense(20, activation='tanh', kernel_regularizer = regularizers.l2(0.02)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='RMSprop', loss='binary_crossentropy', metrics=['accuracy',mean_pred])
    print(model.summary())
    batch_size = 110
    epochs = 10
    
    model_history = model.fit(x_ros, y_ros, validation_data=(Xtest, Ytest),epochs=epochs, batch_size=batch_size, verbose=1)
    y_pred = model.predict_classes(Xtest)
    
scores = model.evaluate(Xtest, Ytest, verbose=0)
cvscores.append(scores[1] * 100)

plot_model_history(model_history)
print("-----RandomOverSampling-----")
print("LSTM Accuracy Score -> ",(np.mean(cvscores)))
print(classification_report(Ytest, y_pred))
print(confusion_matrix(Ytest,y_pred))


In [ ]:
 
cvscores = []
for train, test in cv.split(token):  
    #print("TRAIN:", train, "Test:", test)
    Xtrain, Xtest = token[train], token[test]
    Ytrain, Ytest = y[train], y[test]
    #smote
    from imblearn.over_sampling import SMOTE
    sm = SMOTE(random_state=2)
    X_train_res, y_train_res = sm.fit_sample(Xtrain, Ytrain)    
    
    seed = 121
    np.random.seed(seed)
    model = Sequential()
    model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False, input_length = X_train_res.shape[1]))
    model.add(LSTM(180, activation='tanh', return_sequences=True))
    #model.add(Dropout(0.4))
    model.add(LSTM(180, activation='tanh', return_sequences=True))
    model.add(Dropout(0.3))
    model.add(GlobalMaxPooling1D())
    model.add(Dropout(0.5))
    model.add(Dense(20, activation='tanh', kernel_regularizer = regularizers.l2(0.02)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='RMSprop', loss='binary_crossentropy', metrics=['accuracy',mean_pred])
    print(model.summary())
    batch_size = 110
    epochs = 10

    model_history = model.fit(X_train_res, y_train_res, validation_data=(Xtest, Ytest),epochs=epochs, batch_size=batch_size, verbose=1)
    y_pred = model.predict_classes(Xtest)
    
scores = model.evaluate(Xtest, Ytest, verbose=0)
cvscores.append(scores[1] * 100)

plot_model_history(model_history)
print("-----SMOTE-----")
print("LSTM Accuracy Score -> ",(np.mean(cvscores)))
print(classification_report(Ytest, y_pred))
print(confusion_matrix(Ytest,y_pred))


In [ ]:
 
cvscores = []
for train, test in cv.split(token):  
    #print("TRAIN:", train, "Test:", test)
    Xtrain, Xtest = token[train], token[test]
    Ytrain, Ytest = y[train], y[test]
    #ramdomUnderSample
    from imblearn.under_sampling import RandomUnderSampler

    rus = RandomUnderSampler()
    x_rus, y_rus = rus.fit_sample(Xtrain, Ytrain)  
   
    seed = 121
    np.random.seed(seed)
    model = Sequential()
    model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False, input_length = x_rus.shape[1]))
    model.add(LSTM(180, activation='tanh', return_sequences=True))
    #model.add(Dropout(0.4))
    model.add(LSTM(180, activation='tanh', return_sequences=True))
    model.add(Dropout(0.3))
    model.add(GlobalMaxPooling1D())
    model.add(Dropout(0.5))
    model.add(Dense(20, activation='tanh', kernel_regularizer = regularizers.l2(0.02)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='RMSprop', loss='binary_crossentropy', metrics=['accuracy',mean_pred])
    print(model.summary())
    batch_size = 110
    epochs = 10
    
    model_history = model.fit(x_rus, y_rus, validation_data=(Xtest, Ytest),epochs=epochs, batch_size=batch_size, verbose=1)
    y_pred = model.predict_classes(Xtest)
    
scores = model.evaluate(Xtest, Ytest, verbose=0)
cvscores.append(scores[1] * 100)

plot_model_history(model_history)
print("-----RandomUnderSampling-----")
print("LSTM Accuracy Score -> ",(np.mean(cvscores)))
print(classification_report(Ytest, y_pred))
print(confusion_matrix(Ytest,y_pred))

In [ ]:
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=4, random_state=42)

In [ ]:
 
cvscores = []
for train, test in cv.split(token,y):  
    #print("TRAIN:", train, "Test:", test)
    Xtrain, Xtest = token[train], token[test]
    Ytrain, Ytest = y[train], y[test]
   
    seed = 121
    np.random.seed(seed)
    model = Sequential()
    model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False, input_length = Xtrain.shape[1]))
    model.add(LSTM(180, activation='tanh', return_sequences=True))
    #model.add(Dropout(0.4))
    model.add(LSTM(180, activation='tanh', return_sequences=True))
    model.add(Dropout(0.3))
    model.add(GlobalMaxPooling1D())
    model.add(Dropout(0.5))
    model.add(Dense(20, activation='tanh', kernel_regularizer = regularizers.l2(0.02)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='RMSprop', loss='binary_crossentropy', metrics=['accuracy',mean_pred])
    print(model.summary())
    batch_size = 110
    epochs = 10
    
    model_history = model.fit(Xtrain, Ytrain, validation_data=(Xtest, Ytest),epochs=epochs, batch_size=batch_size, verbose=1)
    y_pred = model.predict_classes(Xtest)

scores = model.evaluate(Xtest, Ytest, verbose=0)
cvscores.append(scores[1] * 100)
plot_model_history(model_history)
print("-----StratifiedKFold-----")
print("LSTM Accuracy Score -> ",(np.mean(cvscores)))
print(classification_report(Ytest, y_pred))
print(confusion_matrix(Ytest,y_pred))
